In [1]:
%matplotlib inline
import numpy as np
import pdb
import sklearn.metrics.pairwise as distance
import matplotlib
#matplotlib.use('tkAgg')
import matplotlib.pyplot as plt
import glob
import sys
from matplotlib.ticker import MultipleLocator
import math
import pandas as pd
import time

In [2]:
read = pd.read_csv('u.data', delim_whitespace=True, names=['user_id', 'movie_id', 'rating', 'time'])
read = read.sort_values('time').reset_index(drop=True)
read

,user_id,movie_id,rating,time
0,259,255,4,874724710
1,259,286,4,874724727
2,259,298,4,874724754
3,259,185,4,874724781
4,259,173,4,874724843
...,...,...,...,...
99995,729,689,4,893286638
99996,729,313,3,893286638
99997,729,328,3,893286638
99998,729,748,4,893286638


In [ ]:
# first we use a 75%-25% train-test split
tau_row = read.loc[75000]
tau = tau_row.time
tau

df_train = read[read['time'] <= tau]
df_test = read[read['time'] > tau]
# now construct the embedding table from training data
unique_id = df_train['user_id'].unique()
unique_movie = read['movie_id'].unique()

table = {}
for uid in unique_id:
    table[uid] = []
    for movie in unique_movie:
        check = df_train.loc[(df_train['user_id'] == uid) & (df_train['movie_id'] == movie)]
        if len(check) > 0:
            if check['rating'].iloc[0] >= 3:
                table[uid].append(1)
            else:
                table[uid].append(0)
        else:
            table[uid].append(0)
    table[uid] = np.array(table[uid])
    

In [ ]:
# Now find top k similar users for each user in testing dataset
ks = [1, 2, 4, 8]
mad = {} # mean avg diff for every test row that has user history appearance in training set
for k in ks:
    mad[k] = []
for uid, movie, score in zip(df_test['user_id'], df_test['movie_id'], df_test['rating']):
    # make sure test user is in train user list
    if uid in unique_id:
        usr_vector = table[uid]
        # rank all users in training set by similarity
        other_usrs = {}
        for train_uid in unique_id:
            # check if the training user has seen this movie
            check = df_train.loc[(df_train['user_id'] == train_uid) & (df_train['movie_id'] == movie)]
            if train_uid != uid and len(check) > 0:
                other_usrs[train_uid] = np.dot(usr_vector,table[train_uid])
        if len(other_usrs) == 0:
            # no user in training set has seen this movie, skip
            continue
        other_usrs = dict(sorted(other_usrs.items(), key=lambda item: item[1], reverse=True))
        similar_usrs = list(other_usrs.keys())[:ks[-1]]
        other_scores = []
        for similar_usr in similar_usrs:
            rating = df_train.loc[(df_train['user_id'] == similar_usr) & 
                                  (df_train['movie_id'] == movie)]['rating'].iloc[0]
            other_scores.append(rating)
        for k in ks:
            mad[k].append(abs(np.mean(other_scores[:k]) - score))
    else:
        # user has no history, cannot recommend, skip user
        pass
pdb.set_trace()
pass

--Return--
None
> <ipython-input-22-c37589420d68>(32)<module>()
     29     else:
     30         # user has no history, cannot recommend, skip user
     31         pass
---> 32 pdb.set_trace()
     33 pass



In [13]:
df_test

,user_id,movie_id,rating,time
75005,832,50,3,888260089
75006,832,471,4,888260089
75007,832,181,3,888260089
75008,832,25,2,888260157
75009,640,354,4,888262331
...,...,...,...,...
99995,729,689,4,893286638
99996,729,313,3,893286638
99997,729,328,3,893286638
99998,729,748,4,893286638
